<a href="https://colab.research.google.com/github/LuisEduardoF/RedesNeurais_202501/blob/main/02_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠🤖 Brincando mais com o Neurônio de McCulloch-Pitts

## Preparação

Copie a classe `MCPNeuron` construída na aula anterior para a célula abaixo.

In [9]:
# resposta:

class MCPNeuron:
    """McCulloch-Pitts Neuron model

    Simulates a biologic neuron using a set of weights and
    an activation threshold.

    """

    def __init__(self, weights: list[float], bias: float):
        self.weights = weights
        self.bias = bias


    def forward(self, x: list[float]):
        assert len(x) == len(self.weights), "Tamanho errado"

        res = 0
        for i in range(len(x)):
          res += self.weights[i] * x[i]

        return int(res >= self.bias)


In [2]:
!python -m pip install requests Pillow opencv-python

## Atividade: Implementando Circuitos

Nesta atividade, vamos construir a nossa primeira **Rede Neural**, isto é, um conjunto de neurônios conectados que realizam uma ou mais tarefas.

Considere o circuito exibido na Figura 1. O objetivo desta atividade é implementar este circuito utilizando apenas neurônios MCP.

Crie uma classe **MCPCircuit** para representar o circuito. A classe deve possuir os seguintes métodos:
- **construtor**: deve inicializar atributos privados do tipo MCPNeuron para representar as portas lógicas.
- **forward**: deve receber uma lista contendo os valores de A, B, C e D, nesta ordem, e retornar a saída do circuito.
  - Adicione um *assert* para verificar se a entrada possui tamanho 4.
  - Os valores de A e B devem ser usados como entrada para o neurônio AND e o valor de C deve ser usado como entrada para o neurônio NOT. As saídas destes neurônios devem ser usadas como entrada para o neurônio OR. E assim por diante.
  - Lembre-se de utilizar *type hints* para informar os tipos de entrada e saída dos métodos.


![circuito](_assets/circuito-logica-digital.png)

Figura 1: Circuito lógico.

In [10]:
import itertools
import pandas as pd

# resposta:

class MCPCircuit():
    def __init__(self):
        self.and_neuron = MCPNeuron([1, 1], 2)
        self.or_neuron = MCPNeuron([1, 1], 1)
        self.not_neuron = MCPNeuron([-1], 0)

    def forward(self, x):
      and_1 = self.and_neuron.forward([x[0], x[1]])
      not_1 = self.not_neuron.forward([x[2]])
      or_1 = self.or_neuron.forward([and_1, not_1])
      and_2 = self.and_neuron.forward([x[3], or_1])
      return self.not_neuron.forward([and_2])


def truth_table(circuit: MCPCircuit) -> pd.DataFrame:
    combinations = list(itertools.product([0, 1], repeat=4))
    table = [c + (circuit.forward(c), ) for c in combinations]
    return pd.DataFrame(table, columns='a b c d result'.split())

circuit = MCPCircuit()
truth_table(circuit) # deve exibir a tabela verdade

,a,b,c,d,result
0,0,0,0,0,1
1,0,0,0,1,0
2,0,0,1,0,1
3,0,0,1,1,1
4,0,1,0,0,1
5,0,1,0,1,0
6,0,1,1,0,1
7,0,1,1,1,1
8,1,0,0,0,1
9,1,0,0,1,0


## Atividade: De volta ao XOR

Na aula passada, vimos que um neurônio MCP é incapaz de implementar a porta lógica XOR.

Inspirados pelo exercício anterior, surge a questão: será que uma rede neural composta por sequências de neurônios MCP é capaz de implementar o XOR? E a resposta é **SIM**!

A Figura 2 mostra um circuito capaz de implementar o XOR usando portas lógicas AND, NOT e OR.

Crie uma classe similar à anterior para reproduzir o circuito.

Esta atividade demonstra um resultado importante: **redes neurais com sequências de duas ou mais camadas contendo funções de ativação não-lineares (como a função degrau utilizada até agora), são capazes de representar funções não lineares**!

![xor circuit](_assets/XOR-gate-circuit.jpg)

Figura 2: Circuito XOR.

In [12]:
import itertools
import pandas as pd

# resposta:

class XorCircuit:
    def __init__(self):
        self.and_neuron = MCPNeuron([1, 1], 2)
        self.or_neuron = MCPNeuron([1, 1], 1)
        self.not_neuron = MCPNeuron([-1], 0)

    def forward(self, x):
      not_1 = self.not_neuron.forward([x[0]])
      not_2 = self.not_neuron.forward([x[1]])

      and_1 = self.and_neuron.forward([x[0], not_2])
      and_2 = self.and_neuron.forward([x[1], not_1])

      or_1 = self.or_neuron.forward([and_1, and_2])
      return or_1

def truth_table(circuit: MCPCircuit) -> pd.DataFrame:
    combinations = list(itertools.product([0, 1], repeat=2))
    table = [c + (circuit.forward(c), ) for c in combinations]
    return pd.DataFrame(table, columns='a b result'.split())

circuit = XorCircuit()
truth_table(circuit) # deve exibir a tabela verdade do XOR

,a,b,result
0,0,0,0
1,0,1,1
2,1,0,1
3,1,1,0
